# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
! pip install yfinance numpy pandas scipy XlsxWriter ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the Nifty 50.

These constituents change over time, so in an ideal world you would connect directly to the index provider and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~50 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
stocks = pd.read_csv('/content/ind_nifty50list.csv')
stocks.rename(columns={'Symbol': 'Ticker'}, inplace=True)
stocks['Ticker'] = stocks['Ticker'] + '.NS'
stocks

,Company Name,Industry,Ticker,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT.NS,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS.NS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP.NS,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT.NS,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK.NS,EQ,INE238A01034
5,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO.NS,EQ,INE917I01010
6,Bajaj Finance Ltd.,Financial Services,BAJFINANCE.NS,EQ,INE296A01024
7,Bajaj Finserv Ltd.,Financial Services,BAJAJFINSV.NS,EQ,INE918I01026
8,Bharat Electronics Ltd.,Capital Goods,BEL.NS,EQ,INE263A01024
9,Bharat Petroleum Corporation Ltd.,Oil Gas & Consumable Fuels,BPCL.NS,EQ,INE029A01011


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [5]:
my_columns = ['Ticker', 'Stock Price', '1 Year Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all Nifty 50 stocks and store their data in the DataFrame using a `for` loop.

In [7]:
import time
from IPython.display import clear_output
import pandas as pd
import numpy as np
import yfinance as yf

# Initialize empty DataFrame
final_dataframe = pd.DataFrame()

# Loop through all stock tickers
for stock in stocks['Ticker']:
    retries = 3  # Number of retries for each stock
    success = False

    while retries > 0 and not success:
        try:
            # Fetch stock data
            data = yf.Ticker(stock).info

            # Extract data
            yr1Return = data.get('52WeekChange', np.NaN)
            stock_price = data.get('currentPrice', np.NaN)

            # Create stock dictionary
            stock_dict = {
                'Ticker': [stock],
                'Stock Price': [stock_price],
                '1 Year Return': [yr1Return],
                'Number of Shares to Buy': ['N/A']
            }

            # Convert to DataFrame and append
            df = pd.DataFrame(stock_dict)
            final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

            # Clear output and print progress
            clear_output(wait=True)
            print(final_dataframe)

            success = True  # Mark as successful
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            retries -= 1  # Reduce retry count
            time.sleep(5)  # Wait before retrying

    time.sleep(2)  # Throttle requests


           Ticker  Stock Price  1 Year Return Number of Shares to Buy
0     ADANIENT.NS      2564.60      -0.134604                     N/A
1   ADANIPORTS.NS      1199.55       0.026265                     N/A
2   APOLLOHOSP.NS      7298.35       0.284469                     N/A
3   ASIANPAINT.NS      2335.95      -0.291793                     N/A
4     AXISBANK.NS      1084.90      -0.033066                     N/A
5   BAJAJ-AUTO.NS      8965.70       0.283776                     N/A
6   BAJFINANCE.NS      7407.25      -0.042496                     N/A
7   BAJAJFINSV.NS      1701.10       0.002800                     N/A
8          BEL.NS       291.95       0.590142                     N/A
9         BPCL.NS       296.40       0.302857                     N/A
10  BHARTIARTL.NS      1598.85       0.524093                     N/A
11   BRITANNIA.NS      4834.10      -0.066298                     N/A
12       CIPLA.NS      1511.25       0.188323                     N/A
13   COALINDIA.NS   

In [8]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,ADANIENT.NS,2564.60,-0.134604,N/A
1,ADANIPORTS.NS,1199.55,0.026265,N/A
2,APOLLOHOSP.NS,7298.35,0.284469,N/A
3,ASIANPAINT.NS,2335.95,-0.291793,N/A
4,AXISBANK.NS,1084.90,-0.033066,N/A
5,BAJAJ-AUTO.NS,8965.70,0.283776,N/A
6,BAJFINANCE.NS,7407.25,-0.042496,N/A
7,BAJAJFINSV.NS,1701.10,0.002800,N/A
8,BEL.NS,291.95,0.590142,N/A
9,BPCL.NS,296.40,0.302857,N/A


In [9]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
15,DUMMYITC.NS,NaN,NaN,N/A


Filling the `NULL` cells of the `1 Year Return` column with mean value

In [10]:
final_dataframe['1 Year Return'].fillna(final_dataframe['1 Year Return'].mean(), inplace=True)

<ipython-input-10-7bedb8488220>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_dataframe['1 Year Return'].fillna(final_dataframe['1 Year Return'].mean(), inplace=True)


In [11]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
15,DUMMYITC.NS,NaN,0.16086,N/A


In [12]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,ADANIENT.NS,2564.60,-0.134604,N/A
1,ADANIPORTS.NS,1199.55,0.026265,N/A
2,APOLLOHOSP.NS,7298.35,0.284469,N/A
3,ASIANPAINT.NS,2335.95,-0.291793,N/A
4,AXISBANK.NS,1084.90,-0.033066,N/A
5,BAJAJ-AUTO.NS,8965.70,0.283776,N/A
6,BAJFINANCE.NS,7407.25,-0.042496,N/A
7,BAJAJFINSV.NS,1701.10,0.002800,N/A
8,BEL.NS,291.95,0.590142,N/A
9,BPCL.NS,296.40,0.302857,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the Nifty 50.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [13]:
final_dataframe.sort_values('1 Year Return', ascending=False, inplace=True)

In [14]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [15]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,TRENT.NS,7307.70,1.400256,N/A
1,M&M.NS,3190.55,0.974167,N/A
2,BEL.NS,291.95,0.590142,N/A
3,BHARTIARTL.NS,1598.85,0.524093,N/A
4,SUNPHARMA.NS,1849.65,0.417031,N/A
5,SHRIRAMFIN.NS,3048.35,0.387916,N/A
6,TECHM.NS,1689.45,0.374710,N/A
7,EICHERMOT.NS,5310.75,0.367252,N/A
8,HCLTECH.NS,1946.65,0.344464,N/A
9,WIPRO.NS,294.45,0.307650,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [16]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [17]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [20]:
for i in range(len(final_dataframe.index)):
    # Check if 'Stock Price' is not NaN before calculation
    if pd.notna(final_dataframe.loc[i, 'Stock Price']):
        final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
            position_size / final_dataframe.loc[i, 'Stock Price'])
    else:
        # Handle NaN 'Stock Price', e.g., set 'Number of Shares to Buy' to 0
        final_dataframe.loc[i, 'Number of Shares to Buy'] = 0

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is that low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Here `hqm` stands for `high-quality momentum`.

In [21]:
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    '1 Year Return',
    '1 Year Return Percentile',
    '6 Months Return',
    '6 Months Return Percentile',
    '3 Months Return',
    '3 Months Return Percentile',
    '1 Month Return',
    '1 Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [23]:
import time
from IPython.display import clear_output
import pandas as pd
import numpy as np
import yfinance as yf

# Initialize the DataFrame
hqm_dataframe = pd.DataFrame()

# Loop through each stock ticker
for stock in stocks['Ticker']:
    retries = 3  # Number of retries for each stock
    success = False

    while retries > 0 and not success:
        try:
            # Fetch stock data
            data = yf.Ticker(stock).info

            # Extract 1-year return
            yr1Return = data.get('52WeekChange', np.NaN)
            stock_price = data.get('currentPrice', np.NaN)

            # Calculate 6-month return
            try:
                hist = yf.Ticker(stock).history(period='6mo')
                mnth6Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
            except:
                mnth6Return = np.NaN

            # Calculate 3-month return
            try:
                hist = yf.Ticker(stock).history(period='3mo')
                mnth3Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
            except:
                mnth3Return = np.NaN

            # Calculate 1-month return
            try:
                hist = yf.Ticker(stock).history(period='1mo')
                mnth1Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
            except:
                mnth1Return = np.NaN

            # Create a stock dictionary
            stock_dict = {
                'Ticker': [stock],
                'Stock Price': [stock_price],
                'Number of Shares to Buy': ['N/A'],
                '1 Year Return': [yr1Return],
                '1 Year Return Percentile': ['N/A'],
                '6 Months Return': [mnth6Return],
                '6 Months Return Percentile': ['N/A'],
                '3 Months Return': [mnth3Return],
                '3 Months Return Percentile': ['N/A'],
                '1 Month Return': [mnth1Return],
                '1 Month Return Percentile': ['N/A'],
                'HQM Score': ['N/A']
            }

            # Append to the DataFrame
            df = pd.DataFrame(stock_dict)
            hqm_dataframe = pd.concat([hqm_dataframe, df], ignore_index=True)

            # Clear output and print progress
            clear_output(wait=True)
            print(hqm_dataframe['Ticker'])

            success = True  # Mark as successful
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            retries -= 1  # Reduce retry count
            time.sleep(5)  # Wait before retrying

    # Throttle requests to prevent hitting the rate limit
    time.sleep(2)

0       ADANIENT.NS
1     ADANIPORTS.NS
2     APOLLOHOSP.NS
3     ASIANPAINT.NS
4       AXISBANK.NS
5     BAJAJ-AUTO.NS
6     BAJFINANCE.NS
7     BAJAJFINSV.NS
8            BEL.NS
9           BPCL.NS
10    BHARTIARTL.NS
11     BRITANNIA.NS
12         CIPLA.NS
13     COALINDIA.NS
14       DRREDDY.NS
15      DUMMYITC.NS
16     EICHERMOT.NS
17        GRASIM.NS
18       HCLTECH.NS
19      HDFCBANK.NS
20      HDFCLIFE.NS
21    HEROMOTOCO.NS
22      HINDALCO.NS
23    HINDUNILVR.NS
24     ICICIBANK.NS
25           ITC.NS
26    INDUSINDBK.NS
27          INFY.NS
28      JSWSTEEL.NS
29     KOTAKBANK.NS
30            LT.NS
31           M&M.NS
32        MARUTI.NS
33          NTPC.NS
34     NESTLEIND.NS
35          ONGC.NS
36     POWERGRID.NS
37      RELIANCE.NS
38       SBILIFE.NS
39    SHRIRAMFIN.NS
40          SBIN.NS
41     SUNPHARMA.NS
42           TCS.NS
43    TATACONSUM.NS
44    TATAMOTORS.NS
45     TATASTEEL.NS
46         TECHM.NS
47         TITAN.NS
48         TRENT.NS
49    ULTRACEMCO.NS


In [24]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ADANIENT.NS,2564.60,N/A,-0.134604,N/A,-19.628946,N/A,-17.695761,N/A,2.004620,N/A,N/A
1,ADANIPORTS.NS,1199.55,N/A,0.026265,N/A,-20.593782,N/A,-15.832859,N/A,-6.914211,N/A,N/A
2,APOLLOHOSP.NS,7298.35,N/A,0.284469,N/A,18.035232,N/A,5.682055,N/A,2.410708,N/A,N/A
3,ASIANPAINT.NS,2335.95,N/A,-0.291793,N/A,-20.016614,N/A,-25.699237,N/A,-5.404145,N/A,N/A
4,AXISBANK.NS,1084.90,N/A,-0.033066,N/A,-15.176775,N/A,-7.723053,N/A,-6.514431,N/A,N/A
5,BAJAJ-AUTO.NS,8965.70,N/A,0.283776,N/A,-4.846962,N/A,-24.061000,N/A,-2.140405,N/A,N/A
6,BAJFINANCE.NS,7407.25,N/A,-0.042496,N/A,2.045107,N/A,-0.357824,N/A,10.962554,N/A,N/A
7,BAJAJFINSV.NS,1701.10,N/A,0.002800,N/A,6.708901,N/A,-11.095435,N/A,5.849045,N/A,N/A
8,BEL.NS,291.95,N/A,0.590142,N/A,-6.918661,N/A,4.754216,N/A,-6.456262,N/A,N/A
9,BPCL.NS,296.40,N/A,0.302857,N/A,-0.230204,N/A,-15.035119,N/A,0.730669,N/A,N/A


In [25]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
15,DUMMYITC.NS,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [26]:
hqm_dataframe['1 Year Return'].fillna(hqm_dataframe['1 Year Return'].mean(), inplace=True)

<ipython-input-26-9a169c6efe90>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hqm_dataframe['1 Year Return'].fillna(hqm_dataframe['1 Year Return'].mean(), inplace=True)


In [27]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
15,DUMMYITC.NS,NaN,N/A,0.16086,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


In [28]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ADANIENT.NS,2564.60,N/A,-0.134604,N/A,-19.628946,N/A,-17.695761,N/A,2.004620,N/A,N/A
1,ADANIPORTS.NS,1199.55,N/A,0.026265,N/A,-20.593782,N/A,-15.832859,N/A,-6.914211,N/A,N/A
2,APOLLOHOSP.NS,7298.35,N/A,0.284469,N/A,18.035232,N/A,5.682055,N/A,2.410708,N/A,N/A
3,ASIANPAINT.NS,2335.95,N/A,-0.291793,N/A,-20.016614,N/A,-25.699237,N/A,-5.404145,N/A,N/A
4,AXISBANK.NS,1084.90,N/A,-0.033066,N/A,-15.176775,N/A,-7.723053,N/A,-6.514431,N/A,N/A
5,BAJAJ-AUTO.NS,8965.70,N/A,0.283776,N/A,-4.846962,N/A,-24.061000,N/A,-2.140405,N/A,N/A
6,BAJFINANCE.NS,7407.25,N/A,-0.042496,N/A,2.045107,N/A,-0.357824,N/A,10.962554,N/A,N/A
7,BAJAJFINSV.NS,1701.10,N/A,0.002800,N/A,6.708901,N/A,-11.095435,N/A,5.849045,N/A,N/A
8,BEL.NS,291.95,N/A,0.590142,N/A,-6.918661,N/A,4.754216,N/A,-6.456262,N/A,N/A
9,BPCL.NS,296.40,N/A,0.302857,N/A,-0.230204,N/A,-15.035119,N/A,0.730669,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

In [29]:
time_periods = [
    '1 Year',
    '6 Months',
    '3 Months',
    '1 Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[change_col],
                                                                           hqm_dataframe.loc[row, change_col]) / 100

In [30]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ADANIENT.NS,2564.60,N/A,-0.134604,0.098039,-19.628946,NaN,-17.695761,NaN,2.004620,NaN,N/A
1,ADANIPORTS.NS,1199.55,N/A,0.026265,0.352941,-20.593782,NaN,-15.832859,NaN,-6.914211,NaN,N/A
2,APOLLOHOSP.NS,7298.35,N/A,0.284469,0.745098,18.035232,NaN,5.682055,NaN,2.410708,NaN,N/A
3,ASIANPAINT.NS,2335.95,N/A,-0.291793,0.039216,-20.016614,NaN,-25.699237,NaN,-5.404145,NaN,N/A
4,AXISBANK.NS,1084.90,N/A,-0.033066,0.215686,-15.176775,NaN,-7.723053,NaN,-6.514431,NaN,N/A
5,BAJAJ-AUTO.NS,8965.70,N/A,0.283776,0.72549,-4.846962,NaN,-24.061000,NaN,-2.140405,NaN,N/A
6,BAJFINANCE.NS,7407.25,N/A,-0.042496,0.176471,2.045107,NaN,-0.357824,NaN,10.962554,NaN,N/A
7,BAJAJFINSV.NS,1701.10,N/A,0.002800,0.27451,6.708901,NaN,-11.095435,NaN,5.849045,NaN,N/A
8,BEL.NS,291.95,N/A,0.590142,0.960784,-6.918661,NaN,4.754216,NaN,-6.456262,NaN,N/A
9,BPCL.NS,296.40,N/A,0.302857,0.784314,-0.230204,NaN,-15.035119,NaN,0.730669,NaN,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [31]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [32]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ADANIENT.NS,2564.60,N/A,-0.134604,0.098039,-19.628946,NaN,-17.695761,NaN,2.004620,NaN,NaN
1,ADANIPORTS.NS,1199.55,N/A,0.026265,0.352941,-20.593782,NaN,-15.832859,NaN,-6.914211,NaN,NaN
2,APOLLOHOSP.NS,7298.35,N/A,0.284469,0.745098,18.035232,NaN,5.682055,NaN,2.410708,NaN,NaN
3,ASIANPAINT.NS,2335.95,N/A,-0.291793,0.039216,-20.016614,NaN,-25.699237,NaN,-5.404145,NaN,NaN
4,AXISBANK.NS,1084.90,N/A,-0.033066,0.215686,-15.176775,NaN,-7.723053,NaN,-6.514431,NaN,NaN
5,BAJAJ-AUTO.NS,8965.70,N/A,0.283776,0.72549,-4.846962,NaN,-24.061000,NaN,-2.140405,NaN,NaN
6,BAJFINANCE.NS,7407.25,N/A,-0.042496,0.176471,2.045107,NaN,-0.357824,NaN,10.962554,NaN,NaN
7,BAJAJFINSV.NS,1701.10,N/A,0.002800,0.27451,6.708901,NaN,-11.095435,NaN,5.849045,NaN,NaN
8,BEL.NS,291.95,N/A,0.590142,0.960784,-6.918661,NaN,4.754216,NaN,-6.456262,NaN,NaN
9,BPCL.NS,296.40,N/A,0.302857,0.784314,-0.230204,NaN,-15.035119,NaN,0.730669,NaN,NaN


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [33]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)

In [34]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [35]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ADANIENT.NS,2564.60,N/A,-0.134604,0.098039,-19.628946,NaN,-17.695761,NaN,2.004620,NaN,NaN
1,ADANIPORTS.NS,1199.55,N/A,0.026265,0.352941,-20.593782,NaN,-15.832859,NaN,-6.914211,NaN,NaN
2,APOLLOHOSP.NS,7298.35,N/A,0.284469,0.745098,18.035232,NaN,5.682055,NaN,2.410708,NaN,NaN
3,ASIANPAINT.NS,2335.95,N/A,-0.291793,0.039216,-20.016614,NaN,-25.699237,NaN,-5.404145,NaN,NaN
4,AXISBANK.NS,1084.90,N/A,-0.033066,0.215686,-15.176775,NaN,-7.723053,NaN,-6.514431,NaN,NaN
5,BAJAJ-AUTO.NS,8965.70,N/A,0.283776,0.72549,-4.846962,NaN,-24.061000,NaN,-2.140405,NaN,NaN
6,BAJFINANCE.NS,7407.25,N/A,-0.042496,0.176471,2.045107,NaN,-0.357824,NaN,10.962554,NaN,NaN
7,BAJAJFINSV.NS,1701.10,N/A,0.002800,0.27451,6.708901,NaN,-11.095435,NaN,5.849045,NaN,NaN
8,BEL.NS,291.95,N/A,0.590142,0.960784,-6.918661,NaN,4.754216,NaN,-6.456262,NaN,NaN
9,BPCL.NS,296.40,N/A,0.302857,0.784314,-0.230204,NaN,-15.035119,NaN,0.730669,NaN,NaN


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [36]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [37]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

200000.0

In [39]:
for i in range(len(hqm_dataframe.index)):
    # Check if 'Stock Price' is not NaN before calculation
    if pd.notna(hqm_dataframe.loc[i, 'Stock Price']):
        hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
            position_size / hqm_dataframe.loc[i, 'Stock Price'])
    else:
        # Handle NaN 'Stock Price', e.g., set 'Number of Shares to Buy' to 0
        hqm_dataframe.loc[i, 'Number of Shares to Buy'] = 0

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [40]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Formatting Our Excel Output

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

# Updated to rupee formatting
rupee_template = writer.book.add_format(
    {
        'num_format': '₹0.00',  # Rupee symbol
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


In [42]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', rupee_template],  # Updated to rupee_template
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)


## Saving Our Excel Output

In [43]:
writer.close()